<a href="https://colab.research.google.com/github/Foxy1987/neuroGLM/blob/master/HowTo_fit_filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning: GLMs
__Content creator:__ David Fox


Hello. I developed a library of python modules which supports fitting and analysis of trial-based time series data in sensory neuroscience using a generalized linear model (GLM). The time series could represent neural spike responses or behavioral responses to arbitrary stimulus presentations. 

For example, we record spike responses from a single neuron or the behavioral response of an animal to multiple presentations of a stimulus, and would like to know what aspects of the stimulus are encoded in the neural response or behavior. This code package allows us to discover such dependencies.

Response variables, here neural recordings or behavior, are are accompanied by various other controlled manipulations or measurements, here an olfactory stimulus. It is hard to find a representation of the stimulus (called a regressor) in the response for regression-style analysis because they don't have an instantaneous effect on the response (dependent) variable. This package allows you to expand and transform your experimental variables (stimulus) to a feature space as a design matrix that is amenable to regression. This allows you to find out the relationship between the response variable and each regressor as a function of time.

This tutorial will explain how to import trial-based experimental data, and build appropriate features spaces to do regression analysis. The ultimate output of the regression is a set of weights on the coefficients that define a stimulus filter (and a post-spike filter)

The filter weights of the linear regression model can be estimated in one of two ways:
1.   Least squares (ridge-regression with cross-validation)
2.   maximum-likelihood estimation under a Poissson GLM model with an exponential nonlinearity





# Import package and libraries

In [31]:
#@title Clone the github repository 
!git clone https://github.com/Foxy1987/neuroGLM
%cd neuroGLM


Cloning into 'neuroGLM'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 117 (delta 49), reused 88 (delta 33), pack-reused 0
Receiving objects: 100% (117/117), 87.63 KiB | 7.30 MiB/s, done.
Resolving deltas: 100% (49/49), done.
/content/neuroGLM/neuroGLM/neuroGLM


In [32]:
#@title add package to google colab
import sys, os
sys.path.append(os.getcwd())

In [33]:
#@title import modules
from glmtools.make_xdsgn import DesignMatrix, Experiment, DesignSpec
import utils.read as io
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from glmtools.fit import neg_log_lik

# Upload files from your local drive to google colab: 

These files should be the following:
1.   stimulus file: this is a regular txt file containing a single column representing the stimulus waveform
2.   spike times file: a .mat file containing a cell array. Each cell is a trial and should contain an array of the times a neuron spiked in response to that stimulus.



In [29]:
from google.colab import files
uploaded = files.upload()

In [16]:
stim, sptimes = io.load_spk_times('stim.txt', 'pn1SpTimes_reverseChirp.mat', 5, 30)

# Bin the spike times
Now we will bin the spike times using the same time bin size used for the stimulus. We will use the time bin centers to bin the spike train.


In [24]:
trials = len(sptimes)
duration = 25
nt = len(stim)
t = (np.arange(nt))*dt
stim = (stim / np.max(stim))*0.01
dt = 0.001                                # 1 ms time bins = 1 KHz sample frequency
sp_count_fun = lambda x: np.histogram(x, np.arange(0.5, len(stim)+1) * dt - dt)[0] 
sps = list(map(sp_count_fun, sptimes))    # apply the binning function sp_count_fun to each trial
stim_ = np.tile(stim, (trials, 1))        # we need a stim for every trial


# Create an experiment object 
Now we will make an experiment object to hold response variable and to register the types of regressors in the experiment

In [ ]:
# make an Experiment object
expt = Experiment(dt, duration, stim=stim_, sptimes=sps)

# register continuous external regressor
expt.registerContinuous('stim')

# register internal sptrain regressor
expt.register_spike_train('sptrain')

# lets visualize our stimulus and response

In [ ]:
sps_ = np.empty((len(stim), trials))
actual = list(map(sp_count_fun, sptimes))
actual_ = list(map(lambda x: x * (np.arange(len(stim)) + 1) * dt, actual))
for i in range(5):
	sps_[:, i] = actual_[i].T

# Share a X axis with each column of subplots
fig, (ax1, ax2) = plt.subplots(2, 1, sharex='col')
ax1.plot(t, stim)
ax2.eventplot(sps_.T, colors='k', linewidth=0.5)



# Create an Experiment object
Now we will make an experiment object to hold response variable and to register the types of regressors in the experiment

In [27]:
# make an Experiment object
expt = Experiment(dt, duration, stim=stim_, sptimes=sps)

# register continuous external regressor
expt.registerContinuous('stim')

# register internal sptrain regressor
expt.register_spike_train('sptrain')

# Create a design specification object
This object holds all the information about how to construct the design matrix for each regressor. The function compile_design_matrix_from_trial_indices() stacks design matrices for a given regressor and for each trial on top of one another. 

In [28]:
trial_inds = list(range(len(sptimes)))
dspec = DesignSpec(expt, trial_inds)

# add stim and spike history regressors
dspec.addRegressorContinuous()
dspec.addRegressorSpTrain()

# Build the design matrix and view the contents
Now we use the information about each regressor included in the experiment and the number of trials to construct a design matrix

In [30]:
dm, X, y = dspec.compileDesignMatrixFromTrialIndices()

forming design matrix from trial indices
forming design matrix from trial indices
convolving padded stimulus with raised cosine basis functions
convolving padded stimulus with raised cosine basis functions
forming design matrix from trial indices
forming design matrix from trial indices
convolving padded stimulus with raised cosine basis functions
convolving padded stimulus with raised cosine basis functions
forming design matrix from trial indices
forming design matrix from trial indices
convolving padded stimulus with raised cosine basis functions
convolving padded stimulus with raised cosine basis functions
forming design matrix from trial indices
forming design matrix from trial indices
convolving padded stimulus with raised cosine basis functions
convolving padded stimulus with raised cosine basis functions
forming design matrix from trial indices
forming design matrix from trial indices
convolving padded stimulus with raised cosine basis functions
convolving padded stimulus with 